The following procedure is used for training a random forest model to later predict the labels for the test data set. 

- The 20 melspectrogram coefficients were previously generated in preprocessing.ipynb from the training data. 

- We trained a random forest classifier on these 20 melspectrogram coefficients. 

- We then predicted the test set labels from the trained random forest model. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

mel_features = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/mel_features.csv')

FileNotFoundError: ignored

In [0]:
mel_features.head()

,Unnamed: 0,fname,label,manually_verified,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0,00044347.wav,Hi-hat,0,0.689457,0.964398,2932.545269,2333.474199,5830.976574,0.111680,36.648372,24.939315,-22.085232,13.273696,-1.400674,0.286512,-2.392393,4.240376,-0.944297,7.417186,-2.185398,3.059307,6.500753,12.706463,3.736702,5.870387,-0.400490,4.164804,-1.698175,3.853791
1,1,001ca53d.wav,Saxophone,1,0.204406,0.923229,3013.667471,2878.744502,6674.518573,0.036462,73.882078,27.773626,-10.746949,11.037612,-4.838018,4.092238,-5.164023,0.537712,-7.852136,-1.915030,-8.089819,-0.549957,-5.423865,0.538680,-5.573924,0.016405,-2.839659,5.608781,0.220564,4.010988
2,2,002d256b.wav,Trumpet,0,0.764434,0.991074,1057.516532,1393.707990,2653.683954,0.005345,-247.694102,29.715195,0.742903,8.938042,6.261368,9.837749,4.959741,7.573261,4.506092,10.080269,8.504948,10.903479,7.870696,9.693287,4.800791,7.440450,4.874798,7.284901,4.745445,7.908377
3,3,00353774.wav,Cello,1,0.279727,0.927143,3245.598952,2805.883303,6702.844426,0.057695,76.192768,25.364872,-12.743592,11.450991,-7.044229,2.298590,-5.045685,3.625585,0.352085,6.339332,-0.660011,4.499685,-4.508909,-0.846468,-4.571769,0.493590,-4.572783,3.684966,0.125284,4.653927
4,4,003b91e8.wav,Cello,0,0.396491,0.956235,2566.340154,2814.470641,6052.079104,0.021268,116.398614,74.268143,-3.818645,19.433300,3.240486,8.229486,-3.693745,3.349545,-2.469640,4.359043,-3.767127,1.091875,-5.481384,0.457637,-4.913501,1.221969,-3.885958,1.972538,-4.278274,-0.143118


In [0]:
#### Be Careful around this block: the 'label' column of the df is being mapped from strings to numbers 
labels = mel_features['label'].unique()
d_label = dict(zip(labels, range(0,10)))
reverse_label = dict(zip(range(0,10), labels))
mel_features['label'] = mel_features['label'].map(lambda x: d_label[x], na_action = 'ignore')

In [0]:
mel_features.head()

,Unnamed: 0,fname,label,manually_verified,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0,00044347.wav,0,0,0.689457,0.964398,2932.545269,2333.474199,5830.976574,0.111680,36.648372,24.939315,-22.085232,13.273696,-1.400674,0.286512,-2.392393,4.240376,-0.944297,7.417186,-2.185398,3.059307,6.500753,12.706463,3.736702,5.870387,-0.400490,4.164804,-1.698175,3.853791
1,1,001ca53d.wav,1,1,0.204406,0.923229,3013.667471,2878.744502,6674.518573,0.036462,73.882078,27.773626,-10.746949,11.037612,-4.838018,4.092238,-5.164023,0.537712,-7.852136,-1.915030,-8.089819,-0.549957,-5.423865,0.538680,-5.573924,0.016405,-2.839659,5.608781,0.220564,4.010988
2,2,002d256b.wav,2,0,0.764434,0.991074,1057.516532,1393.707990,2653.683954,0.005345,-247.694102,29.715195,0.742903,8.938042,6.261368,9.837749,4.959741,7.573261,4.506092,10.080269,8.504948,10.903479,7.870696,9.693287,4.800791,7.440450,4.874798,7.284901,4.745445,7.908377
3,3,00353774.wav,3,1,0.279727,0.927143,3245.598952,2805.883303,6702.844426,0.057695,76.192768,25.364872,-12.743592,11.450991,-7.044229,2.298590,-5.045685,3.625585,0.352085,6.339332,-0.660011,4.499685,-4.508909,-0.846468,-4.571769,0.493590,-4.572783,3.684966,0.125284,4.653927
4,4,003b91e8.wav,3,0,0.396491,0.956235,2566.340154,2814.470641,6052.079104,0.021268,116.398614,74.268143,-3.818645,19.433300,3.240486,8.229486,-3.693745,3.349545,-2.469640,4.359043,-3.767127,1.091875,-5.481384,0.457637,-4.913501,1.221969,-3.885958,1.972538,-4.278274,-0.143118


In [0]:
features = mel_features.iloc[:, 4:].to_numpy()
labels = mel_features['label'].to_numpy()
print(labels)

[0 1 2 ... 5 7 0]


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.10, random_state = 42)
# np.shape(X_train)

In [0]:
clf=RandomForestClassifier(n_estimators=1000)
clf.fit(features, labels)
y_pred=clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [0]:
pip install cleanlab

In [0]:
from cleanlab.classification import LearningWithNoisyLabels
from sklearn.linear_model import LogisticRegression

In [0]:
lnl = LearningWithNoisyLabels(clf=RandomForestClassifier(n_estimators=100))
lnl.fit(X = X_train, s = y_train)
y_pred = lnl.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6884057971014492


In [0]:
test_mel_features = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_testdata.csv')
test_features = test_mel_features.iloc[:, 2:].to_numpy()
test_prediction = clf.predict(test_features)
test_prediction
test_mel_features['label'] = test_prediction


In [0]:
test_mel_features.head()
final_predictions = test_mel_features[['filename', 'label']]
final_predictions = final_predictions.rename(columns={'filename': 'fname'})
final_predictions = final_predictions.set_index('fname')
final_predictions['label'] = final_predictions['label'].apply(lambda x: reverse_label[x])

final_predictions.to_csv('/content/drive/Shared drives/301FP/Data_Folder/RandomForestPredictions1.csv')
final_predictions.head()


,label
fname,
7132bfb1.wav,Trumpet
05ad10fb.wav,Trumpet
f819b0d1.wav,Violin_or_fiddle
fd6430d6.wav,Fart
27ebc5bc.wav,Cello
